In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.transforms import Compose
from torch.utils.data import DataLoader
from time import time

%cd ..
from models import ConvolutionalRNN, ConvolutionalTransformer
from preprocess import compute_normalization, create_transform, invert_log_filter_spectrogram, invert_mel_spectrogram
from evaluate import compute_peaks, compute_predictions, f_measure, evaluate_model
%cd notebooks

import os
from pathlib import Path

import matplotlib.pyplot as plt
import IPython

/Users/runarfosse/Library/Python/3.11/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/runarfosse/Desktop/OneDrive - University of Bergen/Master Thesis
/Users/runarfosse/Desktop/OneDrive - University of Bergen/Master Thesis/notebooks


/Users/runarfosse/Library/Python/3.11/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Verify the spectrogram format for ADTOF-YT.

## First we load both some validation datasets.

In [2]:
# Path to the data directory
data_path = Path(os.getcwd()).parent / "data"

slakh_validation = torch.load(data_path / "slakh2100_flac_redux" / "slakh_validation.pt")
adtof_validation = torch.load(data_path / "adtof" / "adtof_yt_validation.pt")

## For verification, we are guaranteed Slakh is stored in logarithmically filtered spectrograms.
This because we parsed them ourselves.

### Let's listen to a proper reconstruction of one example.

In [3]:
# Pick an example from the dataset
slakh_spectrogram = slakh_validation[1000][0].squeeze(dim=(0, -1)).T

slakh_waveform_log = invert_log_filter_spectrogram(slakh_spectrogram)
IPython.display.Audio(slakh_waveform_log.numpy(), rate=44100)

### Let's now listen to an improper reconstruction instead (treating it like a mel-spectrogram)

In [4]:
slakh_waveform_mel = invert_mel_spectrogram(slakh_spectrogram)
IPython.display.Audio(slakh_waveform_mel.numpy(), rate=44100)

## Now, let's do the same, but for ADTOF-YT.

### First, reconstruct it assuming it's a logarithmically filtered spectrogram.

In [5]:
# Pick an example from the dataset
adtof_spectrogram = adtof_validation[1000][0].squeeze(dim=(0, -1)).T

adtof_waveform_log = invert_log_filter_spectrogram(adtof_spectrogram)
IPython.display.Audio(adtof_waveform_log.numpy(), rate=44100)

### Now, let's treat it like a mel-spectrogram instead.

In [6]:
adtof_waveform_mel = invert_mel_spectrogram(adtof_spectrogram)
IPython.display.Audio(adtof_waveform_mel.numpy(), rate=44100)